In [1]:
%pylab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import pandas as pd
from sqlalchemy import create_engine 
from __future__ import division

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
engine = create_engine('postgresql://postgres@localhost:5432/postgres')

In [3]:
instrument = 'EUR_USD'
fromDate = '2017-05-18 00:00:00+00'
toDate = '2017-05-20 00:00:00+00'
period = '5min'
fromtbl = 'oanda_%s_m1' % (instrument)

win_agg_group_sql = '''
  with intervals as (
  select start, start + interval '%s' as end
  from generate_series('%s', '%s', interval '%s') as start
  )
  select distinct
  intervals.start as dt,
  min(ask_l) over w as low,
  max(ask_h) over w as high,
  first_value(ask_o) over w as open,
  last_value(ask_c) over w as close,
  sum(v) over w as volume
  from
  intervals
  join %s tmb on
  tmb.dt >= intervals.start and
  tmb.dt < intervals.end
  window w as (partition by intervals.start)
  order by intervals.start;
''' % (period,fromDate,toDate,period,fromtbl)


In [246]:
inst_df = pd.read_sql_query(win_agg_group_sql, 
                       con=engine)
inst_df.dt = pd.to_datetime(inst_df.dt,utc=True).dt.tz_convert('Europe/Moscow')
inst_df.set_index(['dt'])
inst_df['dt']=inst_df.dt.dt.strftime('%m/%d/%Y %H:%M:%S')

In [247]:
inst_df['21ew']=inst_df['close'].ewm(span=21, min_periods=21).mean()
inst_df['13ew']=inst_df['close'].ewm(span=13, min_periods=13).mean()
inst_df['55ew']=inst_df['close'].ewm(span=55, min_periods=55).mean()

In [195]:
inst_df[['close','21ew','55ew']].plot(grid=True,figsize=(8,5))

In [250]:
ax=inst_df[['close','13ew','55ew']].plot(grid=True,figsize=(8,5))

In [185]:
SD=0.00015
inst_df['21-55ew']=inst_df['21ew'] - inst_df['55ew']
inst_df['Regime']=np.where(inst_df['21-55ew']>SD,1,0)
inst_df['Regime']=np.where(inst_df['21-55ew']<SD,-1,inst_df['Regime'])
inst_df['Regime'].value_counts()

-1    281
 1    205
 0     54
Name: Regime, dtype: int64

In [190]:
SD=0.00015
inst_df['13-55ew']=inst_df['13ew'] - inst_df['55ew']
inst_df['Regime13']=np.where(inst_df['13-55ew']>SD,1,0)
inst_df['Regime13']=np.where(inst_df['13-55ew']<SD,-1,inst_df['Regime13'])
inst_df['Regime13'].value_counts()

-1    247
 1    239
 0     54
Name: Regime13, dtype: int64

In [192]:
inst_df['Regime13'].plot(lw=1.5)

In [ ]:
(inst_df.dt.max() - inst_df.dt.min()).days 

In [ ]:
(inst_df['Regime'].value_counts())[0]                             